In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import wget
import os
!wget 'http://vis-www.cs.umass.edu/lfw/lfw.tgz'

In [ ]:
!pip install mxnet
!pip install onnxruntime
!pip install insightface

In [ ]:
import numpy as np
import cv2
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import pickle
import os
from os.path import isfile, join, exists

dataset='lfw'
datapath='/content/gdrive/MyDrive/lfw/'
imagepath=datapath
infopath=datapath+'pairs.txt'
modelName='buffalo_s'
ModelRootPath='/content/gdrive/MyDrive/insightface'
imageShape=(250, 250)
outfileName='/content/gdrive/MyDrive/'+modelName+'_'+dataset+'.pickle'

def inference(img):
    app = FaceAnalysis(name=modelName,root=ModelRootPath)
    app.prepare(ctx_id=0, det_size=(640,640))
    faces = app.get(img)

    EM=[]
    if(len(faces)==0):
      EM=np.zeros((512,))

    elif(len(faces)>1):
      max_score=0;
      max_idx=0
      for i in range(len(faces)):
        if(faces[i]['det_score']>max_score):
          max_score=faces[i]['det_score']
          max_idx=i

      EM=faces[max_idx]['embedding']

    else:
      EM=faces[0]['embedding']

    return EM

pairs=[]
with open(infopath, 'r') as f:
    for line in f.readlines()[1:]:
        pair = line.strip().split()
        pairs.append(pair)

fileName=[]
labels=[]
features=[]
i=0

def get_img_num(name,number):
  finalName=''
  if(int(number)<10):
    finalName=name+'_000'+number+'.jpg'
  elif(int(number)<100):
    finalName=name+'_00'+number+'.jpg'
  elif(int(number)<1000):
    finalName=name+'_0'+number+'.jpg'
  else:
    finalName=name+'_'+number+'.jpg'

  return finalName

if(exists(outfileName)):
  with open(outfileName, "rb") as f:
    data = pickle.load(f)
    fileName=data['fileName']
    labels=data['labels']
    features=data['features']
    i=len(fileName)*2+1
    f.close()

for i in range(len(pairs)):
  fileN=[]
  F=np.zeros((2,512))
  lbl=1
  if(len(pairs[i])==3):
    img1=get_img_num(pairs[i][0],pairs[i][1])
    img2=get_img_num(pairs[i][0],pairs[i][2])
    img_path1=imagepath+pairs[i][0]+'/'+img1
    img_path2=imagepath+pairs[i][0]+'/'+img2

  elif(len(pairs[i])==4):
    img1=get_img_num(pairs[i][0],pairs[i][1])
    img2=get_img_num(pairs[i][2],pairs[i][3])
    img_path1=imagepath+pairs[i][0]+'/'+img1
    img_path2=imagepath+pairs[i][2]+'/'+img2
    lbl=0

    fileN=[img1,img2]

    img1=cv2.imread(img_path1)
    img2=cv2.imread(img_path2)
    F[0,:]=inference(img1)
    F[1,:]=inference(img2)

  labels.append(lbl)
  fileName.append(fileN)
  features.append(F)
  featureSet={'labels':labels,'fileName':fileName, 'features':features}
  
  if(exists(outfileName)):
    os.remove(outfileName)
    
  with open(outfileName, 'wb') as f:
    pickle.dump(featureSet, f)

  print(f'file {len(features)}')

featureSet={'labels':labels,'fileName':fileName, 'features':features}
os.remove(outfileName)
with open(outfileName, 'wb') as f:
    pickle.dump(featureSet, f)